In [1]:
pip install Flask pandas matplotlib


  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/3d/44/c4218d351df907c9a39add2d93594ede085a6dc53b5760c67f1eca7871bf/matplotlib-3.9.1-cp312-cp312-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/51/03/36ecc6fd1d018336625ad91b91fd34cd9ba8da5d36f5e1167dfe3275070a/contourpy-1.2.1-cp312-cp312-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/dd/bd/cb8fd2dddd68089c112bf42a88afe188b8ace73f94406539857dcc9347a6/fonttools-4.53.1-cp312-cp312-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency infor

In [3]:
from flask import Flask, render_template
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64

app = Flask(__name__)

def load_data():
    df = pd.read_csv('Latitude Cards.csv')
    return df

def generate_pie_chart(data):
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.pie(data['unique_cardholder_count'], labels=data['card_count'], autopct='%1.1f%%')
    ax.set_title('Cardholder Distribution by Card Count')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    return base64.b64encode(buf.getvalue()).decode('utf-8')

def generate_bar_plot(data, title):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(data['Bank Name'], data['card_id_count'], color='skyblue')
    ax.set_xlabel('Bank Name')
    ax.set_ylabel('Card ID Count')
    ax.set_title(title)
    plt.xticks(rotation=45, ha='right')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    return base64.b64encode(buf.getvalue()).decode('utf-8')

@app.route('/')
def index():
    df = load_data()
    num_unique_cardholders = df['cardholder_id'].nunique()
    num_unique_cards = df['card_id'].nunique()
    new_df = df.groupby('cardholder_id')['card_id'].count().reset_index(name='card_id_count')
    value_counts = new_df['card_id_count'].value_counts().reset_index()
    value_counts.columns = ['card_count', 'unique_cardholder_count']

    pie_chart_data = generate_pie_chart(value_counts)

    df['issuer_bin'] = int(4)
    new_df = df.groupby('issuer_bin')['card_id'].nunique().reset_index(name='card_id_count')
    new_df = new_df.sort_values(by='card_id_count', ascending=False)
    df["issuer_bin"] = df["issuer_bin"].astype(str)
    df["first_1_digits"] = df["issuer_bin"].str[:1]
    grouped_df = df.groupby("first_1_digits")["card_id_count"].sum().reset_index()
    grouped_df = grouped_df.sort_values(by="card_id_count", ascending=False).reset_index(drop=True)
    bank_mapping = {
        '4980': 'Sumitomo Mitsui Card Company Limited',
        '4708': 'YES BANK, LTD.',
        '4297': 'RAKUTEN KC CO., LTD.',
        '4537': 'WELLS FARGO BANK, N.A.',
        '4205': 'AEON CREDIT SERVICE CO., LTD.',
        '4534': 'DC CARD CO., LTD.',
        '4541': 'REDIT SAISON CO., LTD.',
        '4363': 'UNITED COMMERCIAL BANK',
        '4649': 'YAMAGIN CREDIT CO., LTD.',
        '4616': 'U.S. BANK N.A. ND',
        '4986': 'MITSUBISHI UFJ FINANCIAL GROUP, INC.',
        '4539': 'OSTGIROT BANK AB',
        '4097': 'CAJA AHORROS GERONA',
        '4924': 'BANK OF AMERICA, N.A.',
        '4987': 'YAMAGIN CREDIT CO., LTD.',
        '4162': 'UNKNOWN',
        '4721': 'WELLS FARGO BANK IOWA, N.A.',
        '4538': 'MITSUBISHI UFJ FINANCIAL GROUP, INC.',
        '4984': 'BANCO DO BRASIL, S.A.',
        '4122': 'UNITED BANK, LTD.',
        '4901': 'OMC CARD, INC.',
        '4': 'Latitude Bank'
    }
    grouped_df['Bank Name'] = grouped_df['first_1_digits'].map(bank_mapping)
    grouped_df = grouped_df[['first_1_digits', 'Bank Name', 'card_id_count']].head(20)

    bar_plot_data = generate_bar_plot(grouped_df, "Top Issuers by Card ID Count")

    return render_template('index.html', 
                           num_unique_cardholders=num_unique_cardholders, 
                           num_unique_cards=num_unique_cards,
                           pie_chart_data=pie_chart_data,
                           bar_plot_data=bar_plot_data)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/shadeer/Desktop/HOME/AI/Notebooks/kaggle/faiss-rag/ragenv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/shadeer/Desktop/HOME/AI/Notebooks/kaggle/faiss-rag/ragenv/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/shadeer/Desktop/HOME/AI/Notebooks/kaggle/faiss-rag/ragenv/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shadeer/Desktop/HOME/AI/Notebooks/kaggle/faiss-rag/ragenv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
 

SystemExit: 1